# An Analysis of CitiBikes in New York City

All of the codes and images form this report can be found at https://github.com/pmhalvor/Hello_World_II/tree/master/Python/Citibikes

## Introduction

This report will analyze a data set on the CitiBikes in New York City from 2013 until the end of 2016. The three main questions that will be answered here are:
  * What is the distribution of the lengths of time spent per trip?
  * What is the most popular route of the system?`
  * Were any new stations added throughout the time period?

To answer these questions, I'll be using BigQuery over at Google's Cloud Platform for pulling the specific attributes I need to answer these questions, as well as some Python code for plotting and handling the extracted data. 

The data set I'll be using is one of BigQuery's internal datasets, and can be found at http://www.tiny.cc/nyccitibikesgoogle. It contains 16 attributes and over 30 million entries. The four attributes I'll need to answer the questions above are <code>tripduration, start_station, end_station </code> and <code>starttime</code>. 




## Distribution



### SQL


The only attribute needed for analyzing the distribution of the amount of time spent per trip is the <code>tripduration</code> column. This column contains an integer datatype representing the total number of secodns each trip took. To simplfy the data without losing too much of the informaiton hidden within it, I spilt the entries into intervals of 15, 30 and 60 seconds. An example of the SQL query used to do this is shown below:
```
    SELECT
      (tripduration - MOD(tripduration, 15)) / 15 as qmin_interval,

    COUNT(*) as num trips

    FROM ‘bigquery-public-data.new york.citibike trips’

    ORDER BY
      qmin_interval ASC
    LIMIT
      15000

```

On the Google Cloud Platform, visualization of data can easily be done by transferring the results from your query to Google Data Studio. A comma-seperated values file can also be exported if you want to plot your own graphs using other languages like Python or R. I chose to do the latter, using Python to help find the curve that best fit the data.


### Python

In [0]:
# Setting up for the rest of the code
import matplotlib.pyplot as plt
import scipy.stats as stats
import pandas as pd
import numpy as np


There were a few steps getting from .csv file to a fitted distribution plot. The first step was to read the data in to the program. This is done with help from the <code>pandas</code> method <code>read_csv(filename)</code>. I also defined two new variables ``` qmin_interval ```  and ```num_trips```.

In [0]:
trip_data = pd.read_csv("trip_per_quarter_minute.csv") 
qmin_interval = trip_data["qmin_interval"] 
num_trips = trip_data["num_trips"]



Since I wanted to find a matching probability for the data, I needed to convert the values in the ```num_trips``` column to percentages. This was done by simply dividing the ```num_trips``` array by the sum of the same column. 



In [0]:
prob_trips = num_trips/sum(num_trips)

The next step was to plot the data and see what trends show up. After some trail and error, I concluded that the distribution followed a *log-normal distribution* with σ ≈ 0.75. The plot of ```scipy.stats```' ``` lognorm(x, s, loc, scale)``` function on top of the data from my .csv file is shown below.




In [0]:
plt.bar(qmin_interval, prob_trips, label='my data')
plt.xlim(left=0, right=300)
plt.title("Probabilty Distribution")
plt.show()


![](https://raw.githubusercontent.com/pmhalvor/Hello_World_II/master/Python/Citibikes/match_lognorm_quarter.png) 

As you can see, the log-normal function fits pretty nicely, with the largest error around 20-30 minutes. A peak occurs at 6 minutes (remember from the SQL query that this plot will show intervals of 15 seconds). The plot is also truncated, since there were relatively few entries with durations longer than 75 minutes.  


## Most popular route

### SQL

This next objective is to find the most popular route among the bikers. The columns needed to find this are ```start_station``` and ```end_station```. A ```UNION``` or ```CONCAT``` operation is need in the query in order to count the number of trips per *route*. I chose to join the starting and ending stations names using ```CONCAT``` to include a whitespace between the station names. Here is the SQL-query used:




```
  SELECT  
    CONCAT(start_station_name, ' to ', end_station_name) as route,
    COUNT(*) as num_trips   FROM  `bigquery-public-data.new_york.citibike_trips`
  
  GROUP BY  
    start_station_name,  
    end_station_name
  
  ORDER BY  
    num_trips DESC 

  LIMIT  
    1000
```



As a result, I found the most popular route was picking up at Central Park S & 6th Avenue and delivering at the same place. Actually, the top four most popular routes delivered the bike to the same location as pick-up. 

<img src="https://raw.githubusercontent.com/pmhalvor/Hello_World_II/master/Python/Citibikes/popular_routes.jpg" width="73%" />

This made me curious about how long these rolls through Central Park usually took. I combined some of the query from the previous two tasks with my current to find the distribution of trip durations for pick ups and drop offs at Central Park and 6th. The SQL query is shown below: 


```
SELECT
  (tripduration - MOD(tripduration, 60)) / 60 as cp_route_times,
  COUNT(*) as num_trips
  
FROM
  `bigquery-public-data.new_york.citibike_trips`
  
WHERE start_station_name="Central Park S & 6 Ave" AND end_station_name='Central Park S & 6 Ave'
    
GROUP BY 
  1
  
ORDER BY
  cp_route_times ASC

LIMIT
  1000
```



These results were saved as ```cp_route_times.csv```. 

### Python

Now you can plot this data in Python by running the code below.

In [0]:
data = pd.read_csv("cp_route_times.csv")
time = data['cp_route_times']
num_trips = data['num_trips']

plt.bar(time[:180], num_trips[:180])
plt.xlim(left=0, right=180)
plt.xlabel("Minutes Spent on bike")
plt.ylabel("Amount of riders")
plt.show()

It turns out, these trips averaged much longer, with 20-40 minutes as the most frequent durations, and a peak at 28 minutes. The plot blow shows the distribution.

![](https://raw.githubusercontent.com/pmhalvor/Hello_World_II/master/Python/Citibikes/trip_duration_cp.jpg)

## New Stations

### SQL

To find out if any new stations were added to the CitiBike system over the span of the data set, I pulled the name of every start station connect with the year of that trip. Since the data set spanned over a 4 year period, this would be an easy way to see if any new stations were added. Here is an example of the SQL query used:




```
SELECT  
  start_station_name as names,  
  SUBSTR(STRING(TIMESTAMP_TRUNC(starttime, YEAR)), 0, 4) as years,  
  COUNT(*) as num_trips   FROM  `bigquery-public-data.new_york.citibike_trips`
 
GROUP BY  
  years,  
  names 

ORDER BY  
  names ASC,  
  years ASC 
  
LIMIT  
  2000
```



The results showed 14 new stations after one year and an additional 152 the next year. By 2016, the number of stations had nearly doubled from 2013, the first year included in the data set. I generated two graphs with the help of Google's Data Studio to help visualize the change.

![alt text](https://raw.githubusercontent.com/pmhalvor/Hello_World_II/master/Python/Citibikes/number_of_stations_bar.jpg)
![alt text](https://raw.githubusercontent.com/pmhalvor/Hello_World_II/master/Python/Citibikes/number_of_stations_donut.jpg)

##### Note, I only checked the starting stations. Had there been absolutely no changes, I might have needed to run the same check on the end stations

### Python

With some simple Python code, lists of containing the news of these newly added stations can be created. 

We start by reading in the data

In [0]:
data = pd.read_csv("new_stations.csv")
names = data['names']
years = data['years']

The create your empty lists 

In [0]:
new_2014 = []
new_2015 = []
new_2016 = []

Now you can run a for loop with some if tests to iterate through your lists of stations, to check if it exsisted the year before. If not, it will be added to its corresponding list.

In [0]:
for i in range(len(names)):
	if int(years[i])!=2013:
		if names[i]!=names[i-1]:
			# Station is new
			if int(years[i])==2014:
				new_2014.append(names[i])
			elif int(years[i])==2015:
				new_2015.append(names[i])
			elif int(years[i])==2016:
				new_2016.append(names[i])

The lists created from this code snippet should look something like this:
![](https://raw.githubusercontent.com/pmhalvor/Hello_World_II/master/Python/Citibikes/new_stations_short.jpg)

By checking the lengths of these lists, it can be concluded that not only were new stations added as the yaers went on, but there were also stations that were removed from the system. This is shown by the longer lengths of the newly added stations lists than the differences in total start stations form our graphs above. In 2015 and  2016 were there respectively 13 and 20 removed from the system. In 2014, there were no stations removed since the difference between the previous year and the total number of new stations match. 

## Summary

We've now seen that the data follows a **log-normal distribution** with a σ ≈ 0.75. We also found that the 4 most popular routes for the entire 4 year period the dataset covers were picked up and dropped off at the same stations,
with **Central Park S and & 6th Avenue** as the **most popular**. Finally, not only were there **new stations added** each year, but there were also stations that were removed from the system. 


The dataset used to answer these questions also contained information on the ages and membership types of each biker, along with the exact longitude and latitude for each stations. There is plenty more information to pull from these data, but I'll save that for the next guy. Thanks for paying attention until this point. I hope this report fulfilled all your expectations! 